MACD, Moving Average Convergence-Divergence, is generated by calculating to exponential moving averages, an exponential moving average is a simple moving average that gives more weightage, momentum depending on the recent price data.

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

import pandas_datareader.data as web
import pandas_ta as ta

import matplotlib.pyplot as plt
from datetime import date

In [5]:
data = pd.read_csv(r'BTC-USD_from_2018-01-01.csv')

macd = ta.macd(data['Close'])
macd

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
1600,-2595.389238,5.326451,-2600.715690
1601,-2512.732215,70.386780,-2583.118995
1602,-2348.220816,187.918543,-2536.139359
1603,-2290.275608,196.691001,-2486.966609


In [6]:
data = pd.concat([data, macd], axis = 1).reindex(data.index)
data

,Date,Open,High,Low,Close,Adj Close,Volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
0,2018-01-01,14112.200195,14112.200195,13154.700195,13657.200195,13657.200195,10291200000,NaN,NaN,NaN
1,2018-01-02,13625.000000,15444.599609,13163.599609,14982.099609,14982.099609,16846600192,NaN,NaN,NaN
2,2018-01-03,14978.200195,15572.799805,14844.500000,15201.000000,15201.000000,16871900160,NaN,NaN,NaN
3,2018-01-04,15270.700195,15739.700195,14522.200195,15599.200195,15599.200195,21783199744,NaN,NaN,NaN
4,2018-01-05,15477.200195,17705.199219,15202.799805,17429.500000,17429.500000,23840899072,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1600,2022-05-20,30311.119141,30664.976562,28793.605469,29200.740234,29200.740234,30749382605,-2595.389238,5.326451,-2600.715690
1601,2022-05-21,29199.859375,29588.869141,29027.394531,29432.226562,29432.226562,17274840442,-2512.732215,70.386780,-2583.118995
1602,2022-05-22,29432.472656,30425.861328,29275.183594,30323.722656,30323.722656,21631532270,-2348.220816,187.918543,-2536.139359
1603,2022-05-23,30309.396484,30590.585938,28975.560547,29098.910156,29098.910156,31483454557,-2290.275608,196.691001,-2486.966609


Basic MACD Strategy

In [11]:
def strategy_MACD(df, risk):
    long_MACD= []
    short_MACD = []
    position = False

    for i in range(0, len(df)):
        if df['MACD_12_26_9'][i] > df['MACDs_12_26_9'][i]:
            short_MACD.append(np.nan)
            if position == False:
                long_MACD.append(df['Adj Close'][i])
                position = True
            else:
                long_MACD.append(np.nan)
        elif df['MACD_12_26_9'][i] < df['MACDs_12_26_9'][i]:
            long_MACD.append(np.nan)
            if position == True:
                short_MACD.append(df['Adj Close'][i])
                position = False
            else:
                short_MACD.append(np.nan)
        elif position == True and df['Adj Close'][i] < df['Adj Close'][i - 1] * (1 - risk):
            short_MACD.append(df['Adj Close'][i])
            short_MACD.append(np.nan)
            position = False 
        else:
            long_MACD.append(np.nan)
            short_MACD.append(np.nan)

    data['MACD_Long_Signal'] = long_MACD
    data['MACD_Short_Signal'] = short_MACD

In [13]:
strategy_MACD = strategy_MACD(data, 0.025)
strategy_MACD

TypeError: 'NoneType' object is not callable

In [14]:
data

,Date,Open,High,Low,Close,Adj Close,Volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,MACD_Long_Signal,MACD_Short_Signal
0,2018-01-01,14112.200195,14112.200195,13154.700195,13657.200195,13657.200195,10291200000,NaN,NaN,NaN,NaN,NaN
1,2018-01-02,13625.000000,15444.599609,13163.599609,14982.099609,14982.099609,16846600192,NaN,NaN,NaN,NaN,NaN
2,2018-01-03,14978.200195,15572.799805,14844.500000,15201.000000,15201.000000,16871900160,NaN,NaN,NaN,NaN,NaN
3,2018-01-04,15270.700195,15739.700195,14522.200195,15599.200195,15599.200195,21783199744,NaN,NaN,NaN,NaN,NaN
4,2018-01-05,15477.200195,17705.199219,15202.799805,17429.500000,17429.500000,23840899072,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1600,2022-05-20,30311.119141,30664.976562,28793.605469,29200.740234,29200.740234,30749382605,-2595.389238,5.326451,-2600.715690,29200.740234,NaN
1601,2022-05-21,29199.859375,29588.869141,29027.394531,29432.226562,29432.226562,17274840442,-2512.732215,70.386780,-2583.118995,NaN,NaN
1602,2022-05-22,29432.472656,30425.861328,29275.183594,30323.722656,30323.722656,21631532270,-2348.220816,187.918543,-2536.139359,NaN,NaN
1603,2022-05-23,30309.396484,30590.585938,28975.560547,29098.910156,29098.910156,31483454557,-2290.275608,196.691001,-2486.966609,NaN,NaN
